Installing plotly

In [1]:
import numpy as np
import pandas as pd
import matplotlib
#needed because of "Cannot get window extent w/o renderer"
matplotlib.use('qt4agg')
import scipy
import seaborn as sns
sns.set_style('whitegrid')

import altair
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["patch.force_edgecolor"] = True
%store -r cleaned

import sys
!/opt/anaconda/bin/conda install --yes --prefix {sys.prefix} plotly

In [2]:
cleaned = cleaned.copy()
cleaned.head()
cleaned.to_csv("digital_divide.csv")

# First, a nice chloropleth map


In [3]:
map_df = cleaned[cleaned['date'] == 2016][['country', 'Code', 'internet_users']]
map_df.head()

,,country,Code,internet_users
country,date,,,
Afghanistan,2016,Afghanistan,AFG,10.595726
Albania,2016,Albania,ALB,66.363445
Algeria,2016,Algeria,DZA,42.945527
Angola,2016,Angola,AGO,13.000000
Antigua and Barbuda,2016,Antigua and Barbuda,ATG,73.000000


In [4]:
original = cleaned.copy()
cleaned.dropna(inplace=True)
cleaned

country  date  Access Electricity  Cellular %  \
country     date                                                      
Afghanistan 2016  Afghanistan  2016           84.137138   62.335417   
            2015  Afghanistan  2015           71.500000   58.420528   
            2014  Afghanistan  2014           89.500000   56.191333   
            2013  Afghanistan  2013           67.259552   52.966473   
            2012  Afghanistan  2012           69.100000   49.972753   
            2011  Afghanistan  2011           43.222019   46.444058   
            2010  Afghanistan  2010           42.700000   35.467766   
            2009  Afghanistan  2009           44.854885   37.494200   
            2003  Afghanistan  2003           11.751966    0.867120   
Albania     2016      Albania  2016          100.000000  115.152265   
            2015      Albania  2015          100.000000  116.337513   
            2014      Albania  2014          100.000000  115.026115   
            2013      Albania  2013          100.000000  126.276491   
            2012      Albania  2012          100.000000  119.861413   
            2011      Albania  2011          100.000000  105.922829   
            2010      Albania  2010          100.000000   91.560929   
            2009      Albania  2009          100.000000   83.160464   
            2008      Albania  2008          100.000000   62.160727   
            2007      Albania  2007          100.000000   76.802494   
            2006      Albania  2006          100.000000   62.530387   
            2005      Albania  2005          100.000000   49.696494   
            2004      Albania  2004          100.000000   40.661487   
            2003      Albania  2003          100.000000   35.358349   
            2002      Albania  2002          100.000000   27.284132   
            2001      Albania  2001          100.000000   12.575230   
            2000      Albania  2000          100.000000    0.954237   
            1999      Albania  1999          100.000000    0.353322   
            1998      Albania  1998          100.000000    0.180426   
            1997      Albania  1997          100.000000    0.106691   
            1996      Albania  1996          100.000000    0.074380   
...                       ...   ...                 ...         ...   
Zambia      2002       Zambia  2002           17.400000    1.250781   
            2001       Zambia  2001           19.498520    1.119721   
            2000       Zambia  2000           16.700000    0.938666   
            1999       Zambia  1999           18.559008    0.275301   
            1998       Zambia  1998           19.000000    0.083013   
            1997       Zambia  1997           17.533768    0.047069   
            1996       Zambia  1996           17.300000    0.028964   
            1995       Zambia  1995           16.435108    0.016931   
Zimbabwe    2016     Zimbabwe  2016           38.145138   79.743884   
            2015     Zimbabwe  2015           33.700000   80.858499   
            2014     Zimbabwe  2014           32.300000   76.556585   
            2013     Zimbabwe  2013           37.076813   90.558714   
            2012     Zimbabwe  2012           36.728878   85.745933   
            2011     Zimbabwe  2011           36.900000   63.948179   
            2010     Zimbabwe  2010           36.063057   54.662975   
            2009     Zimbabwe  2009           43.369082   28.898095   
            2008     Zimbabwe  2008           35.457310   12.204335   
            2007     Zimbabwe  2007           35.184479    9.194766   
            2006     Zimbabwe  2006           37.200000    6.470045   
            2005     Zimbabwe  2005           34.701820    5.000838   
            2004     Zimbabwe  2004           34.481380    3.331987   
            2003     Zimbabwe  2003           34.266785    2.878376   
            2002     Zimbabwe  2002           34.200000    2.710118   
            2001     Zimbabwe  2001           

In [5]:
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

In [6]:
configure_plotly_browser_state()
data = [ 
    dict(
        type = 'choropleth', 
        locations = map_df['Code'], 
        z = map_df['internet_users'], 
        text = map_df['country'], 
        colorscale = "Viridis",
        autocolorscale = False, 
        reversescale = True, 
        marker = dict(
            line = dict(
                color = 'rgb(180, 180, 180)', 
                width = 0.5
            )), 
        colorbar=dict(
            autotick = False, 
            tickprefix = '%',
            title= "Internet Users"), 
        #colorscale='Viridis'
    )]
layout = dict(
    title = "2016 Internet Users in %, Source: World Bank",
    geo = dict(
    showframe=False, 
    showcoastlines = False,
    projection = dict(
        type= "Mercator"
    )
    )
)
fig = dict(data=data, layout = layout)
iplot(fig, validate=False, filename="test")

Now, I want to create an animated graph that goes through the different years

First, I'll use another example to get me familiar

In [7]:
import pandas as pd

init_notebook_mode(connected=True)

years = ['1996', '2001', '2006', '2011', '2016']
regions = []
for region in cleaned['Region']: 
    if region not in regions:
        regions.append(region)
# make figure
figure = {
    'data': [],
    'layout': {},
    'frames': []
}

# fill in most of layout
figure['layout']['xaxis'] = {'range': [-10, 20] , 'title': 'GDP Growth'}
figure['layout']['yaxis'] = {'range': [-10, 110], 
                             'title': 'Internet Users'}
figure['layout']['hovermode'] = 'closest'
figure['layout']['sliders'] = {
    'args': [
        'transition', {
            'duration': 400,
            'easing': 'cubic-in-out'
        }
    ],
    'initialValue': '1952',
    'plotlycommand': 'animate',
    'values': years,
    'visible': True
}
figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Year:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

# make data
year = 1991
for region in regions:
    dataset_by_year = cleaned[cleaned['date'] == year]
    dataset_by_year_and_cont = dataset_by_year[dataset_by_year['Region'] == region]

    data_dict = {
        'x': list(dataset_by_year_and_cont['GDP Growth (annual)']),
        'y': list(dataset_by_year_and_cont['internet_users']),
        'mode': 'markers',
        'text': list(dataset_by_year_and_cont['country']),
        'marker': {
            'sizemode': 'area',
            'sizeref': 100,
            'size': list(dataset_by_year_and_cont['GDP pcp PPP'])
        },
        'name': region
    }
    figure['data'].append(data_dict)
    
# make frames
for year in years:
    frame = {'data': [], 'name': str(year)}
    for region in regions:
        dataset_by_year = cleaned[cleaned['date'] == int(year)]
        dataset_by_year_and_cont = dataset_by_year[dataset_by_year['Region'] == region]

        data_dict = {
            'x': list(dataset_by_year_and_cont['GDP Growth (annual)']),
            'y': list(dataset_by_year_and_cont['internet_users']),
            'mode': 'markers',
            'text': list(dataset_by_year_and_cont['country']),
            'marker': {
                'sizemode': 'area',
                'sizeref': 200,
                'size': list(dataset_by_year_and_cont['GDP pcp PPP'])
            },
            'name': region
        }
        frame['data'].append(data_dict)

    figure['frames'].append(frame)
    slider_step = {'args': [
        [year],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': year,
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)

    
figure['layout']['sliders'] = [sliders_dict]

iplot(figure)

This code from plotly.com works, but my code below does not

In [8]:
from itertools import cycle
import plotly.tools as tls
lines = ["-","--","-."]
linecycler = cycle(lines)
fig = plt.figure()
ax = fig.add_subplot(111)
for i in range(10):
    x = range(i,i+10)
    ax.plot(range(10),x,next(linecycler))

ax.set_title('Cycling through line styles in matplotlib')

plotly_fig = tls.mpl_to_plotly( fig )
iplot(plotly_fig, filename='mpl-cycle-linestyles')



In [9]:
cleaned.head

<bound method NDFrame.head of                       country  date  Access Electricity  Cellular %  \
country     date                                                      
Afghanistan 2016  Afghanistan  2016           84.137138   62.335417   
            2015  Afghanistan  2015           71.500000   58.420528   
            2014  Afghanistan  2014           89.500000   56.191333   
            2013  Afghanistan  2013           67.259552   52.966473   
            2012  Afghanistan  2012           69.100000   49.972753   
            2011  Afghanistan  2011           43.222019   46.444058   
            2010  Afghanistan  2010           42.700000   35.467766   
            2009  Afghanistan  2009           44.854885   37.494200   
            2003  Afghanistan  2003           11.751966    0.867120   
Albania     2016      Albania  2016          100.000000  115.152265   
            2015      Albania  2015          100.000000  116.337513   
            2014      Albania  2014          10

In [10]:

grouped = cleaned.groupby(['date', 'Region'], as_index = False).mean()

fig, ax = plt.subplots()
regions = list(cleaned['Region'].unique())

style={x:y for x, y in zip(regions, ['s-', 'o-', '^-', ':', '--', '-', '-.'])}

for region in regions:
    grouped.loc[grouped['Region']==region].plot(x = 'date', y='internet_users', style=style, ax = ax, label = region)
ax.legend(loc='center left', bbox_to_anchor=(1.3, 0.5),
          fancybox=True, shadow=True)

plotly_fig = tls.mpl_to_plotly( fig )
iplot(plotly_fig, filename='mpl-cycle-linestyles')

/home/toni/.conda/envs/digital_divide/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning:

'date' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version

/home/toni/.conda/envs/digital_divide/lib/python3.6/site-packages/plotly/matplotlylib/renderer.py:390: UserWarning:

Bummer! Plotly can currently only draw Line2D objects from matplotlib that are in 'data' coordinates!

/home/toni/.conda/envs/digital_divide/lib/python3.6/site-packages/plotly/matplotlylib/renderer.py:522: UserWarning:

Looks like the annotation(s) you are trying 
to draw lies/lay outside the given figure size.

Therefore, the resulting Plotly figure may not be 
large enough to view the full text. To adjust 
the size of the figure, use the 'width' and 
'height' keys in the Layout object. Alternatively,
use the Margin object to adjust the figure's margins.

/home/toni/.conda/envs/digital_divide/lib/python3.6/site-packages/plotly/matplotlyl

In [11]:
f, ax = plt.subplots(figsize=(20,10))
ax = sns.swarmplot(x='date', y ='internet_users', data = cleaned)
plt.tight_layout()
plotly_fig = tls.mpl_to_plotly( f)
iplot(plotly_fig, filename='mpl-cycle-linestyles')

In [12]:
import plotly.plotly as py
import plotly.tools as tls

import matplotlib.pyplot as plt
import numpy as np

spread = np.random.rand(50) * 100
center = np.ones(25) * 50
flier_high = np.random.rand(10) * 100 + 100
flier_low = np.random.rand(10) * -100
data = np.concatenate((spread, center, flier_high, flier_low), 0)

mpl_fig = plt.figure()
ax = mpl_fig.add_subplot(111)

ax.boxplot(data)

ax.set_xlabel('Data Points')
ax.set_ylabel('Variance')

plotly_fig = tls.mpl_to_plotly( mpl_fig )
iplot(plotly_fig, filename='boxplot-basic')

ValueError: 
    Invalid value of type 'builtins.str' received for the 'color' property of scatter.marker
        Received value: 'none'

    The 'color' property is a color and may be specified as:
      - A hex string (e.g. '#ff0000')
      - An rgb/rgba string (e.g. 'rgb(255,0,0)')
      - An hsl/hsla string (e.g. 'hsl(0,100%,50%)')
      - An hsv/hsva string (e.g. 'hsv(0,100%,100%)')
      - A named CSS color:
            aliceblue, antiquewhite, aqua, aquamarine, azure,
            beige, bisque, black, blanchedalmond, blue,
            blueviolet, brown, burlywood, cadetblue,
            chartreuse, chocolate, coral, cornflowerblue,
            cornsilk, crimson, cyan, darkblue, darkcyan,
            darkgoldenrod, darkgray, darkgrey, darkgreen,
            darkkhaki, darkmagenta, darkolivegreen, darkorange,
            darkorchid, darkred, darksalmon, darkseagreen,
            darkslateblue, darkslategray, darkslategrey,
            darkturquoise, darkviolet, deeppink, deepskybl

In [ ]:
#py.sign_in('DemoAccount', 'lr1c37zw81')

n = 50
x, y, z, s, ew = np.random.rand(5, n)
c, ec = np.random.rand(2, n, 4)
area_scale, width_scale = 500, 5

fig, ax = plt.subplots()
sc = ax.scatter(x, y, c=c,
                s=np.square(s)*area_scale,
                edgecolor=ec,
                linewidth=ew*width_scale)
ax.grid()
plotly_fig = tls.mpl_to_plotly(mpl_fig)

unique_url = iplot(plotly_fig)